In [64]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [65]:
# Loading data
data_no_menu = pd.read_csv('../data/all.csv')
data_menu = pd.read_csv('../data/all_with_onehot.csv')

In [66]:
data_no_menu.head()

,Date,Weekday,MealType,Paytm+Cash,Coupons,SemType,Holiday,CouponsMand
0,2022-09-01,3,BreakFast,186.0,117.0,Acad,0,0.0
1,2022-09-01,3,Lunch,293.0,217.0,Acad,0,0.0
2,2022-09-01,3,EveningSnacks,37.0,139.0,Acad,0,0.0
3,2022-09-01,3,Dinner,113.0,220.0,Acad,0,0.0
4,2022-09-02,4,BreakFast,100.0,236.0,Acad,0,0.0


In [67]:
# Preprocessing the dataset
def preprocess(data, to_onehot=True, to_drop_vacation=False):
    data = data.dropna()
        
    # Converting the date column to datetime
    data['Date'] = pd.to_datetime(data['Date'])

    # Adding day, dayofweek, month and year columns
    data['Day'] = pd.DatetimeIndex(data['Date']).day
    data['Month'] = pd.DatetimeIndex(data['Date']).month
    data['Year'] = pd.DatetimeIndex(data['Date']).year

    # Scaling the data
    # to_normalize = ['Day']
    # scaler = MinMaxScaler()
    # data[to_normalize] = scaler.fit_transform(data[to_normalize])

    # to_normalize = ['Holiday']
    # scaler = StandardScaler()
    # data[to_normalize] = scaler.fit_transform(data[to_normalize])

    # Encoding the categorical data
    if to_onehot:
        categorical_features = ['Weekday', 'Month', 'Year', 'MealType', 'SemType']
        data = pd.get_dummies(data, columns=categorical_features)
    else:
        # Using label encoding
        categorical_features = ['MealType', 'SemType']
        for feature in categorical_features:
            data[feature] = data[feature].astype('category')
            data[feature] = data[feature].cat.codes

    # Dropping vacations (if required)
    if to_drop_vacation:
        data = data[data['Semtype_Vacation'] == 0]

    # Splitting into X and y
    X = data.drop(columns=['Paytm+Cash', 'Coupons'])
    y_paytm = data['Paytm+Cash']
    y_coupons = data['Coupons']
    y_total = data['Paytm+Cash'] + data['Coupons']

    return X, y_paytm, y_coupons, y_total

In [68]:
# Splitting the data into train and test sets (with vacations)
X_no_onehot, y_paytm_no_onehot, y_coupons_no_onehot, y_total_no_onehot = preprocess(data_no_menu, to_onehot=False)
X_onehot, y_paytm_onehot, y_coupons_onehot, y_total_onehot = preprocess(data_no_menu, to_onehot=True)

# cutoff at 2023-08-31 for no onehot
X_train_no_onehot = X_no_onehot[X_no_onehot['Date'] <= '2023-08-31']
X_test_no_onehot = X_no_onehot[X_no_onehot['Date'] > '2023-08-31']
y_paytm_train_no_onehot = y_paytm_no_onehot[X_no_onehot['Date'] <= '2023-08-31']
y_paytm_test_no_onehot = y_paytm_no_onehot[X_no_onehot['Date'] > '2023-08-31']
y_coupons_train_no_onehot = y_coupons_no_onehot[X_no_onehot['Date'] <= '2023-08-31']
y_coupons_test_no_onehot = y_coupons_no_onehot[X_no_onehot['Date'] > '2023-08-31']
y_total_train_no_onehot = y_total_no_onehot[X_no_onehot['Date'] <= '2023-08-31']
y_total_test_no_onehot = y_total_no_onehot[X_no_onehot['Date'] > '2023-08-31']

# cutoff at 2023-08-31 for onehot
X_train_onehot = X_onehot[X_onehot['Date'] <= '2023-08-31']
X_test_onehot = X_onehot[X_onehot['Date'] > '2023-08-31']
y_paytm_train_onehot = y_paytm_onehot[X_onehot['Date'] <= '2023-08-31']
y_paytm_test_onehot = y_paytm_onehot[X_onehot['Date'] > '2023-08-31']
y_coupons_train_onehot = y_coupons_onehot[X_onehot['Date'] <= '2023-08-31']
y_coupons_test_onehot = y_coupons_onehot[X_onehot['Date'] > '2023-08-31']
y_total_train_onehot = y_total_onehot[X_onehot['Date'] <= '2023-08-31']
y_total_test_onehot = y_total_onehot[X_onehot['Date'] > '2023-08-31']

# Drop the date column
X_train_no_onehot = X_train_no_onehot.drop(columns=['Date'])
X_test_no_onehot = X_test_no_onehot.drop(columns=['Date'])
X_train_onehot = X_train_onehot.drop(columns=['Date'])
X_test_onehot = X_test_onehot.drop(columns=['Date'])

C:\Users\rajad\AppData\Local\Temp\ipykernel_21032\1111604582.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Date'] = pd.to_datetime(data['Date'])
C:\Users\rajad\AppData\Local\Temp\ipykernel_21032\1111604582.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Day'] = pd.DatetimeIndex(data['Date']).day
C:\Users\rajad\AppData\Local\Temp\ipykernel_21032\1111604582.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in

In [69]:
# Defining parameters for grid search
parameters = {
    'n_estimators': [10, 25, 50, 100, 500],
    'max_depth': [5, 10, 20, None],
    'min_samples_split': [2, 4, 6],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', 25, None]
}

In [70]:
def grid_search(X_train, y_train, X_test, y_test, parameters):
   # Grid search for paytm+cash using for loops
    best_parameters = {}

    for n_estimators in parameters['n_estimators']:
        for max_depth in parameters['max_depth']:
            for min_samples_split in parameters['min_samples_split']:
                for min_samples_leaf in parameters['min_samples_leaf']:
                    for max_features in parameters['max_features']:
                        rfr = RandomForestRegressor(n_estimators=n_estimators, 
                                                    max_depth=max_depth, 
                                                    min_samples_split=min_samples_split, 
                                                    min_samples_leaf=min_samples_leaf,
                                                    max_features=max_features, 
                                                    random_state=42)
                        rfr.fit(X_train, y_train)
                        y_pred = rfr.predict(X_test)
                        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
                        r2 = r2_score(y_test, y_pred)
                        y_pred_train = rfr.predict(X_train)
                        rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
                        r2_train = r2_score(y_train, y_pred_train)
                        best_parameters[(rmse, r2, rmse_train, r2_train)] = (n_estimators, max_depth, min_samples_split, min_samples_leaf, max_features)

    # Printing the best parameters with scores
    best_parameters = sorted(best_parameters.items(), key=lambda x: x[0][0])
    print('Best parameters:')
    print('Test RMSE:', best_parameters[0][0][0])
    print('Test R2:', best_parameters[0][0][1])
    print('Train RMSE:', best_parameters[0][0][2])
    print('Train R2:', best_parameters[0][0][3])
    print('Parameters:', best_parameters[0][1])

    return best_parameters[0][1]

In [71]:
# Grid search for paytm+cash using onehot
best_parameters_paytm_onehot = grid_search(X_train_onehot, y_paytm_train_onehot, X_test_onehot, y_paytm_test_onehot, parameters)

Best parameters:
Test RMSE: 17.57900960972621
Test R2: 0.7860533319577394
Train RMSE: 12.659515135914436
Train R2: 0.9080855478469568
Parameters: (25, None, 4, 1, 'log2')


In [72]:
# Grid search for paytm_cash without using onehot
best_parameters_paytm_no_onehot = grid_search(X_train_no_onehot, y_paytm_train_no_onehot, X_test_no_onehot, y_paytm_test_no_onehot, parameters)

Best parameters:
Test RMSE: 11.551095637122748
Test R2: 0.9076231648012663
Train RMSE: 12.839751526471952
Train R2: 0.9054497030755154
Parameters: (500, 10, 6, 1, None)


In [73]:
# Grid search for coupons using onehot
best_parameters_coupons_onehot = grid_search(X_train_onehot, y_coupons_train_onehot, X_test_onehot, y_coupons_test_onehot, parameters)

Best parameters:
Test RMSE: 57.993215930307166
Test R2: 0.6045193816169899
Train RMSE: 16.609273128416362
Train R2: 0.9652106350459195
Parameters: (10, None, 4, 1, 25)


In [74]:
# Grid search for coupons without using onehot
best_parameters_coupons_no_onehot = grid_search(X_train_no_onehot, y_coupons_train_no_onehot, X_test_no_onehot, y_coupons_test_no_onehot, parameters)

Best parameters:
Test RMSE: 54.12950649276085
Test R2: 0.6554605512467115
Train RMSE: 20.59799205999863
Train R2: 0.9464949196827607
Parameters: (10, None, 6, 1, None)


In [75]:
# Grid search for total using onehot
best_parameters_total_onehot = grid_search(X_train_onehot, y_total_train_onehot, X_test_onehot, y_total_test_onehot, parameters)

Best parameters:
Test RMSE: 60.4202852407321
Test R2: 0.7327849392388255
Train RMSE: 14.351042443393283
Train R2: 0.9833637611137053
Parameters: (50, None, 2, 1, 'sqrt')


In [76]:
# Grid search for total without using onehot
best_parameters_total_no_onehot = grid_search(X_train_no_onehot, y_total_train_no_onehot, X_test_no_onehot, y_total_test_no_onehot, parameters)

Best parameters:
Test RMSE: 59.16446864864241
Test R2: 0.7437774631895977
Train RMSE: 17.19554265825008
Train R2: 0.9761152872860931
Parameters: (50, 20, 2, 1, 'log2')


In [77]:
# Using menu
# Splitting the data into train and test sets (with vacations)
X_no_onehot, y_paytm_no_onehot, y_coupons_no_onehot, y_total_no_onehot = preprocess(data_menu, to_onehot=False)
X_onehot, y_paytm_onehot, y_coupons_onehot, y_total_onehot = preprocess(data_menu, to_onehot=True)

# cutoff at 2023-08-31 for no onehot
X_train_no_onehot = X_no_onehot[X_no_onehot['Date'] <= '2023-08-31']
X_test_no_onehot = X_no_onehot[X_no_onehot['Date'] > '2023-08-31']
y_paytm_train_no_onehot = y_paytm_no_onehot[X_no_onehot['Date'] <= '2023-08-31']
y_paytm_test_no_onehot = y_paytm_no_onehot[X_no_onehot['Date'] > '2023-08-31']
y_coupons_train_no_onehot = y_coupons_no_onehot[X_no_onehot['Date'] <= '2023-08-31']
y_coupons_test_no_onehot = y_coupons_no_onehot[X_no_onehot['Date'] > '2023-08-31']
y_total_train_no_onehot = y_total_no_onehot[X_no_onehot['Date'] <= '2023-08-31']
y_total_test_no_onehot = y_total_no_onehot[X_no_onehot['Date'] > '2023-08-31']

# cutoff at 2023-08-31 for onehot
X_train_onehot = X_onehot[X_onehot['Date'] <= '2023-08-31']
X_test_onehot = X_onehot[X_onehot['Date'] > '2023-08-31']
y_paytm_train_onehot = y_paytm_onehot[X_onehot['Date'] <= '2023-08-31']
y_paytm_test_onehot = y_paytm_onehot[X_onehot['Date'] > '2023-08-31']
y_coupons_train_onehot = y_coupons_onehot[X_onehot['Date'] <= '2023-08-31']
y_coupons_test_onehot = y_coupons_onehot[X_onehot['Date'] > '2023-08-31']
y_total_train_onehot = y_total_onehot[X_onehot['Date'] <= '2023-08-31']
y_total_test_onehot = y_total_onehot[X_onehot['Date'] > '2023-08-31']

# Drop the date column
X_train_no_onehot = X_train_no_onehot.drop(columns=['Date'])
X_test_no_onehot = X_test_no_onehot.drop(columns=['Date'])
X_train_onehot = X_train_onehot.drop(columns=['Date'])
X_test_onehot = X_test_onehot.drop(columns=['Date'])

In [78]:
# Grid search for paytm_cash without using onehot
best_parameters_paytm_no_onehot = grid_search(X_train_no_onehot, y_paytm_train_no_onehot, X_test_no_onehot, y_paytm_test_no_onehot, parameters)

Best parameters:
Test RMSE: 11.80221121067723
Test R2: 0.9035630458894336
Train RMSE: 10.81884323716174
Train R2: 0.9328708243200896
Parameters: (50, 10, 4, 1, None)


In [79]:
# Grid search for paytm+cash using onehot
best_parameters_paytm_onehot = grid_search(X_train_onehot, y_paytm_train_onehot, X_test_onehot, y_paytm_test_onehot, parameters)

Best parameters:
Test RMSE: 18.40000401290644
Test R2: 0.7656027265330807
Train RMSE: 14.443283172337722
Train R2: 0.8803585933986943
Parameters: (25, 20, 4, 1, 'log2')


In [80]:
# Grid search for coupons without using onehot
best_parameters_coupons_no_onehot = grid_search(X_train_no_onehot, y_coupons_train_no_onehot, X_test_no_onehot, y_coupons_test_no_onehot, parameters)

Best parameters:
Test RMSE: 52.25100021602408
Test R2: 0.6789593425649789
Train RMSE: 18.85378977119594
Train R2: 0.955172701415597
Parameters: (10, 10, 6, 1, None)


In [81]:
# Grid search for coupons using onehot
best_parameters_coupons_onehot = grid_search(X_train_onehot, y_coupons_train_onehot, X_test_onehot, y_coupons_test_onehot, parameters)

Best parameters:
Test RMSE: 59.81619790345949
Test R2: 0.57926520644265
Train RMSE: 17.713831752947783
Train R2: 0.960429614765103
Parameters: (10, 20, 2, 1, 'sqrt')


In [82]:
# Grid search for total without using onehot
best_parameters_total_no_onehot = grid_search(X_train_no_onehot, y_total_train_no_onehot, X_test_no_onehot, y_total_test_no_onehot, parameters)

Best parameters:
Test RMSE: 53.823435922144604
Test R2: 0.787950024607066
Train RMSE: 22.987223124700016
Train R2: 0.9573163772514232
Parameters: (10, 20, 2, 1, 'sqrt')


In [83]:
# Grid search for total using onehot
best_parameters_total_onehot = grid_search(X_train_onehot, y_total_train_onehot, X_test_onehot, y_total_test_onehot, parameters)

Best parameters:
Test RMSE: 59.86626969059177
Test R2: 0.7376628565169308
Train RMSE: 14.162363307631903
Train R2: 0.9837983325953252
Parameters: (100, None, 2, 1, 25)
